In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
import os
from numpy import *
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/mlproject2/Training Data.csv')
target_data = pd.read_csv('/content/drive/MyDrive/mlproject2/Training Data Target.csv')

In [73]:
data

,user_id,aov,category
0,37327,29128.0,Phones
1,37327,354.0,Fashion
2,37327,460.0,Home Decor
3,37327,761.0,Fashion
4,41260,16658.0,Phones
...,...,...,...
257402,29237,689.0,Fashion
257403,15197,2195.0,Groceries
257404,2114,559.0,Back to School
257405,35741,18451.0,Phones


In [74]:
del data["aov"]

In [75]:
data["category"].value_counts()

Phones                       66341
Fashion                      60327
Home Decor                   29703
Toys                         21106
Groceries                    12810
Back to School               12566
Gaming                       11937
Consumer Durables            11731
Books                         6010
Kitchen cleaning Supplies     3205
Beauty Products               3016
Ereaders                      3012
Board Games                   3001
Painiting Supplies            2985
Pet Supplies                  2955
Fitness                       2945
Laptops                       1888
TVs                           1869
Name: category, dtype: int64

In [76]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']
category_id = list(range(1,19))
# get the list of tuples from two lists.
# and merge them by using zip().
list_of_tuples = list(zip(category, category_id))
df = pd.DataFrame(list_of_tuples,columns = ['category', 'category_id'])
merged_df = pd.merge(data,df,on='category')
del merged_df["category"]
merged_df

,user_id,category_id
0,37327,1
1,41260,1
2,19360,1
3,19360,1
4,19360,1
...,...,...
257402,46282,10
257403,47716,10
257404,11175,10
257405,20009,10


In [77]:
merged_df.insert(2, "freq", list(ones(data.shape[0])), True)
merged_df

,user_id,category_id,freq
0,37327,1,1.0
1,41260,1,1.0
2,19360,1,1.0
3,19360,1,1.0
4,19360,1,1.0
...,...,...,...
257402,46282,10,1.0
257403,47716,10,1.0
257404,11175,10,1.0
257405,20009,10,1.0


In [78]:
freqs = merged_df.groupby(['user_id','category_id'])['freq'].count()
dataset = freqs.to_frame()

In [79]:
dataset.reset_index(inplace=True)
# del table['index']
dataset = dataset.rename(columns={"user_id": "userID", "category_id": "itemID", "freq":"rating"})
dataset

,userID,itemID,rating
0,1,1,1
1,1,2,2
2,1,4,1
3,1,5,1
4,1,16,2
...,...,...,...
159864,50075,1,1
159865,50078,9,1
159866,50079,10,1
159867,50080,2,1


In [80]:
dataset

,userID,itemID,rating
0,1,1,1
1,1,2,2
2,1,4,1
3,1,5,1
4,1,16,2
...,...,...,...
159864,50075,1,1
159865,50078,9,1
159866,50079,10,1
159867,50080,2,1


In [81]:
dataset['rating'].describe()

count    159869.000000
mean          1.610112
std           0.944003
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           9.000000
Name: rating, dtype: float64

In [82]:
pip install scikit-surprise

In [83]:
table = dataset.pivot_table(values='rating', index='userID',columns='itemID')
table

itemID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
userID,,,,,,,,,,,,,,,,,,
1,1.0,2.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,3.0,2.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3.0,4.0,NaN,2.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
7,2.0,2.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50075,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
import surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(dataset[['userID', 'itemID', 'rating']], reader)
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo1 = SVD(n_factors=50, n_epochs=50, biased=True, lr_all = 0.1, random_state=15, verbose=True)
algo1.fit(trainset)
# We can now use this dataset as we please, e.g. calling cross_validate
# svd.fit(trainset)
# Run 5-fold cross-validation and then print results


## Testing Trained model on Target_predictions

In [86]:
del target_data['aov']
target_data

,user_id,category
0,13153,Phones
1,42853,Phones
2,30550,Phones
3,9797,Phones
4,8171,Phones
...,...,...
13240,49383,Fashion
13241,49590,Home Decor
13242,49799,Fitness
13243,49900,Fitness


In [87]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']

item_dict = {}
for i in range(1,19):
  item_dict[i] = category[i-1];
# pred=zeros((len(data), 3),dtype = object)
# for i in range(len(data)):
#   b=sorted(top3[i], key=itemgetter(1),reverse=True)
#   pred[i][0]=item_dict[b[0][0]]
#   pred[i][1]=item_dict[b[1][0]]
#   pred[i][2]=item_dict[b[2][0]]

# pred3 = zeros((len(data)),dtype = object)
# for i in range(len(data)):
#   pred3[i] = pred[i][0] + ", " + pred[i][1] + ", " + pred[i][2]


In [88]:
top1 = [[] for i in range(len(target_data))] 
accuracy = 0
from operator import itemgetter
# Driver Code
for i in range(len(target_data)):
    for iid in range(1,19):
        est = algo1.predict(target_data.iloc[i][0], iid, verbose=False).est
        top1[i].append([iid,est])
    b=sorted(top1[i], key=itemgetter(1),reverse=True)
    if (item_dict[b[0][0]] == target_data.iloc[i].iloc[1]) : 
      accuracy=accuracy+1
  
accuracy = accuracy/(len(target_data))
accuracy

0.3466968667421669

In [89]:
# from sklearn.neighbors import NearestNeighbors
# model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors=5)
# model_knn.fit(table)
# distances, indices = model_knn.kneighbors(table.iloc[1,:].values.reshape(1, -1))
# for i in range(0, len(distances.flatten())):
#         print('{0}: {1}, with distance of {2}:'.format(i, table.index[indices.flatten()[i]], distances.flatten()[i]))

In [90]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# df = data.corr()
# sns.heatmap(df,annot = True,square = True)
# plt.show()

In [91]:
# table = merged_df.pivot_table(values='count', index='user_id',columns='category_id', aggfunc=sum)
# table
top1[0]

[[1, 1.8507491845237471],
 [2, 1.7329643748629624],
 [3, 1.042346120330031],
 [4, 1.160022761033501],
 [5, 1],
 [6, 1],
 [7, 1],
 [8, 1],
 [9, 1],
 [10, 1],
 [11, 1],
 [12, 1],
 [13, 1],
 [14, 1],
 [15, 1],
 [16, 1],
 [17, 1],
 [18, 1]]

## Processing test data

In [92]:
test_data = pd.read_csv('/content/drive/MyDrive/mlproject2/Test Data.csv')
del test_data["Unnamed: 0"]

In [93]:
del test_data["aov"]

In [94]:
test_data["category"].value_counts()

Phones                       44627
Fashion                      40082
Home Decor                   19911
Toys                         14096
Groceries                     8559
Back to School                8405
Gaming                        8109
Consumer Durables             7984
Books                         3968
Ereaders                      2163
Pet Supplies                  2059
Beauty Products               2055
Kitchen cleaning Supplies     2042
Painiting Supplies            1987
Board Games                   1981
Fitness                       1961
Laptops                       1310
TVs                           1255
Name: category, dtype: int64

In [95]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']
category_id = list(range(1,19))
# get the list of tuples from two lists.
# and merge them by using zip().
list_of_tuples = list(zip(category, category_id))
df = pd.DataFrame(list_of_tuples,columns = ['category', 'category_id'])
merged_df = pd.merge(test_data,df,on='category')
del merged_df["category"]
merged_df


,user_id,category_id
0,43323,1
1,43323,1
2,43323,1
3,43323,1
4,4269,1
...,...,...
172549,459,14
172550,3215,14
172551,927,14
172552,927,14


In [96]:
test_data

,user_id,category
0,43323,Phones
1,43323,Phones
2,43323,Phones
3,43323,Phones
4,43323,Fashion
...,...,...
172549,12403,Groceries
172550,31036,Back to School
172551,12652,Fashion
172552,35726,Kitchen cleaning Supplies


In [97]:
merged_df.insert(2, "freq", list(ones(test_data.shape[0])), True)
merged_df

,user_id,category_id,freq
0,43323,1,1.0
1,43323,1,1.0
2,43323,1,1.0
3,43323,1,1.0
4,4269,1,1.0
...,...,...,...
172549,459,14,1.0
172550,3215,14,1.0
172551,927,14,1.0
172552,927,14,1.0


In [98]:
freqs = merged_df.groupby(['user_id','category_id'])['freq'].count()
dataset = freqs.to_frame()

In [99]:
dataset.reset_index(inplace=True)
# del table['index']
dataset = dataset.rename(columns={"user_id": "userID", "category_id": "itemID", "freq":"rating"})
dataset

,userID,itemID,rating
0,2,1,2
1,2,2,2
2,2,12,1
3,2,16,1
4,3,1,3
...,...,...,...
107058,50037,5,1
107059,50050,5,1
107060,50050,6,1
107061,50074,10,1


In [100]:
dataset['rating'].describe()

count    107063.000000
mean          1.611705
std           0.941935
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           8.000000
Name: rating, dtype: float64

In [113]:
import surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [114]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 18))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(dataset[['userID', 'itemID', 'rating']], reader)
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = SVD(n_factors=50,n_epochs=20, biased=True,lr_all = 0.001, random_state=15, verbose=True)

algo.fit(trainset)
# We can now use this dataset as we please, e.g. calling cross_validate
# svd.fit(trainset)
# Run 5-fold cross-validation and then print results
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=4, verbose=True)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=1)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Streaming output truncated to the last 5000 lines.
37525 [5]
37526 [3]
37528 [4]
37530 [4]
37533 [2]
37534 [4]
37535 [4]
37539 [3]
37546 [6]
37552 [4]
37554 [2]
37555 [2]
37560 [4]
37564 [2]
37566 [3]
37570 [7]
37577 [2]
37578 [5]
37579 [3]
37580 [5]
37581 [4]
37582 [5]
37584 [3]
37587 [3]
37590 [4]
37591 [3]
37592 [5]
37596 [4]
37601 [2]
37606 [2]
37607 [4]
37611 [6]
37612 [1]
37614 [3]
37615 [3]
37617 [3]
37619 [3]
37621 [6]
37623 [5]
37625 [6]
37627 [3]
37629 [1]
37630 [3]
37632 [6]
37634 [3]
37637 [6]
37638 [3]
37640 [2]
37642 [4]
37645 [4]
37646 [4]
37648 [6]
37649 [3]
37651 [6]
37655 [1]
37656 [4]
37662 [4]
37664 [6]
37665 [6]
37667 [3]
37668 [6]
37675 [3]
37679 [4]
37680 [5]
37683 [7]
37685 [9]
37686 [3]
37687 [4]
37689 [4]
37690 [4]
37691 [3]
37692 [3]
37693 [1]
37696 [7]
37698 [4]
37699 [6]
37701 [4]
37702 [3]
37706 [2]
37713 [6]
37723 [2]
37727 [6]
37729 [3]
37730 [2]
37732 [7]
37733 [4]
37734 [2]
37736 [4]
37737 [7]
37738 [4]
37739 [3]
37742 [3]
37743 [3]
37744 [4]
37745 [2]

##Creating csv for submission

In [103]:
data = pd.read_csv("/content/drive/MyDrive/mlproject2/Sample Submission.csv")
data

,user_id,pred3
0,43323,"Laptops, Fashion, Consumer Durables"
1,4269,"Laptops, Fashion, Consumer Durables"
2,21731,"Laptops, Fashion, Consumer Durables"
3,14422,"Laptops, Fashion, Consumer Durables"
4,12862,"Laptops, Fashion, Consumer Durables"
...,...,...
19976,42011,"Laptops, Fashion, Consumer Durables"
19977,16846,"Laptops, Fashion, Consumer Durables"
19978,25357,"Laptops, Fashion, Consumer Durables"
19979,49936,"Laptops, Fashion, Consumer Durables"


In [ ]:
data

In [105]:
top3 = [[] for i in range(len(data))] 
top = [[] for i in range(len(data))] 

# Driver Code
for i in range(len(data)):
    for iid in range(1,19):
        est = algo.predict(data.iloc[i][0], iid, verbose=False).est
        top3[i].append([iid,est])

In [106]:
from operator import itemgetter
b=sorted(top3[0], key=itemgetter(1),reverse=True)
b

[[1, 2.478088316604949],
 [2, 2.398824761537372],
 [3, 1.654972653282418],
 [4, 1.498330086059403],
 [7, 1.345985217909622],
 [6, 1.3439419882544197],
 [5, 1.3116473017462027],
 [9, 1.2153632512376125],
 [11, 1.2072835695717263],
 [13, 1.1857213259486767],
 [14, 1.1665010667941582],
 [12, 1.1605709425766504],
 [18, 1.1491821063371337],
 [15, 1.1490731491367727],
 [16, 1.1440295934808506],
 [10, 1.1415383715406644],
 [17, 1.104882463068193],
 [8, 1.0918535836351793]]

In [107]:
del data['pred3']
data

,user_id
0,43323
1,4269
2,21731
3,14422
4,12862
...,...
19976,42011
19977,16846
19978,25357
19979,49936


In [108]:
category = ['Phones', 'Fashion', 'Home Decor', 'Toys','Groceries','Back to School','Gaming','Consumer Durables','Books','Kitchen cleaning Supplies','Beauty Products','Ereaders','Board Games','Painiting Supplies','Pet Supplies','Fitness','Laptops','TVs']

item_dict = {}
for i in range(1,19):
  item_dict[i] = category[i-1]
  
pred=zeros((len(data), 3),dtype = object)
for i in range(len(data)):
  b=sorted(top3[i], key=itemgetter(1),reverse=True)
  pred[i][0]=item_dict[b[0][0]]
  pred[i][1]=item_dict[b[1][0]]
  pred[i][2]=item_dict[b[2][0]]

pred3 = zeros((len(data)),dtype = object)
for i in range(len(data)):
  pred3[i] = pred[i][0] + ", " + pred[i][1] + ", " + pred[i][2]


In [ ]:
from collections import defaultdict
top_n

In [109]:
data['pred3'] = pred3
data 

,user_id,pred3
0,43323,"Phones, Fashion, Home Decor"
1,4269,"Fashion, Phones, Home Decor"
2,21731,"Fashion, Phones, Home Decor"
3,14422,"Phones, Fashion, Home Decor"
4,12862,"Phones, Fashion, Home Decor"
...,...,...
19976,42011,"Phones, Fashion, Home Decor"
19977,16846,"Fashion, Phones, Home Decor"
19978,25357,"Fashion, Phones, Home Decor"
19979,49936,"Phones, Fashion, Home Decor"


In [110]:
# My final Submission 
h=['user_id','pred3']
data.to_csv("recomm.csv",header=h,index=False)